In [1]:
%matplotlib inline

import numpy as np
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from sklearn import manifold
import time
import os
import community as cm
import networkx as nx
from natsort import natsorted
from sklearn import cluster

In [2]:
nets = [nx.read_graphml('hepth/irvine/' + str(i) + '.graphml') for i in range(10)]

In [3]:
def relative_stability(Fs, Gs):
    stabs = []
    for i in range(len(Fs) - 1):
        j1 = np.matrix(Fs[i])
        j2 = np.matrix(Fs[i+1])
                
        j2 = j2[:j1.shape[0], :]
                
        a1 = nx.to_numpy_matrix(Gs[i])
        a2 = nx.to_numpy_matrix(Gs[i+1])[:a1.shape[0], :a1.shape[0]]
        
        t1 = np.linalg.norm(j2 - j1, 'fro')
#         t1 /= np.linalg.norm(j1, 'fro')
        t2 = np.linalg.norm(a2 - a1, 'fro')
#         t2 /= np.linalg.norm(a1, 'fro')
        
        stabs.append(t1 / t2)
        
    absolutes = [abs(x) for x in stabs]
#     return stabs, max(absolutes) - min(absolutes)
    return stabs, np.average(stabs)

In [4]:
def get_k_means_performance(network, embedding):
    opt = cm.best_partition(network)
    mods = []
    opt = len(set(opt.values()))
    for i in range(2, 1+opt):
        pers = []
        for _ in range(5):
            clusters = cluster.KMeans(int(i)).fit_predict(embedding)
            cluster_dict = dict([(list(network.nodes)[j], clusters[j]) for j in range(len(clusters))])
            pers.append(cm.modularity(cluster_dict,network))
        mods.append(sum(pers) / len(pers))
    
    print(mods.index(max(mods)) + 1, max(mods))
    return max(mods)

In [18]:
mods = []
print('Louvain')
for i in tqdm_notebook(range(10)):
    opt = cm.best_partition(nets[i])
    mods.append(cm.modularity(opt, nets[i]))
    print(mods[-1])
print(np.average(mods))

Louvain


0.3578411500979892
0.3538049888886369
0.3458559602676937
0.3436892141208862
0.3449352082206598
0.3323037863298524
0.332593918745295
0.3335187145557656
0.33949155571441336
0.3429593154387838

0.3426993812379976


In [19]:
embs = pickle.load(open('hepth/irvine_node2vec.pkl', 'rb'))
mods = []
print('Node2Vec')
for i in range(len(nets)):
    mods.append(get_k_means_performance(nets[i], embs[i]))
                
print(np.average(mods), mods)

Node2Vec
5 0.3352425300008054
6 0.3305599051667265
6 0.33021512427552235
4 0.3239840668919719
4 0.32754029204975665
5 0.3233156443137621
5 0.32219539631543703
6 0.32572287334593575
6 0.3228142538214752
6 0.32001330899950065
0.3261603395180893 [0.3352425300008054, 0.3305599051667265, 0.33021512427552235, 0.3239840668919719, 0.32754029204975665, 0.3233156443137621, 0.32219539631543703, 0.32572287334593575, 0.3228142538214752, 0.32001330899950065]


In [6]:
(times, embs, total_losses) = pickle.load(open('walk_irvine/' + str(20) + '.pkl', 'rb'))
(times, embs, total_losses) = pickle.load(open('walk_irvine/' + '_a20' + '.pkl', 'rb'))
mods = []
print('WalkEncoder')
for i in range(len(nets)):
    mods.append(get_k_means_performance(nets[i], embs[i]))
                
print(np.average(mods), times)

WalkEncoder
2 0.2634280920293163
2 0.28769096421047907
3 0.2970201493287292
3 0.30040688102639906
2 0.29170254191454836
5 0.28747250702621346
2 0.27499345536721165
2 0.28376941398865785
2 0.23865274803197276
3 0.20423451731202974
0.27293712702355577 [11.247549533843994, 21.269246339797974, 30.95884680747986, 40.604427337646484, 51.5280876159668, 61.421557664871216, 71.30028080940247, 81.51390719413757, 91.77668523788452, 102.51626801490784]


In [26]:
(times, embs) = pickle.load(open('DynamicGEM-master/dynIrvine/150.pkl', 'rb'))
mods = []
print('DynGEM')
for i in range(len(nets)):
    mods.append(get_k_means_performance(nets[i], embs[i]))
                
print(np.average(mods), times)

DynGEM
10 -0.02879500120808613
1 -0.011206818129117103
1 -0.013975257442758182
1 -0.013444293363273824
10 -0.0346745700378583
1 -0.04466567628136825
10 -0.042145426060114996
1 -0.012163327032136174
1 -0.0063288904390336465
1 -0.005392469137928069
-0.021279172913167468 [13.642547845840454, 25.5270938873291, 37.65000295639038, 49.52720928192139, 61.379589796066284, 75.25614643096924, 90.74853110313416, 105.92093181610107, 123.17936539649963, 137.63871335983276]


In [25]:
relative_stability(pickle.load(open('hepth/irvine_node2vec.pkl', 'rb')), nets)

([9.567799147082185,
  11.164295447169211,
  10.790299115510303,
  8.252979360463293,
  13.027527834794235,
  12.243971824645996,
  6.408114069563698,
  2.7185853816217156,
  7.495505615274653],
 9.074341977347252)

In [23]:
relative_stability(pickle.load(open('walk_irvine/20.pkl', 'rb'))[1], nets)

([4.203692999454086,
  3.688900660953094,
  3.258531709054922,
  2.933542736618872,
  3.574260353457497,
  3.752866268157959,
  1.6798242477262322,
  0.6709656949771104,
  2.05633685643073],
 2.8687690585367225)

In [24]:
relative_stability(pickle.load(open('DynamicGEM-master/dynIrvine/150.pkl', 'rb'))[1], nets)

([28.34937489024831,
  26.550328867766115,
  26.717454170470717,
  23.615665622990992,
  29.19827042247263,
  33.31911087036133,
  16.095826966399557,
  5.269090047454109,
  17.696227626013933],
 22.9790388315753)